In [ ]:
import os.path as osp
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline

In [ ]:
def target(x):
    return np.exp(-(x-3)**2) + np.exp(-(3*x-2)**2) + 1/(x**2+1)

In [ ]:
x = np.linspace(-2, 6, 10000).reshape(-1, 1)
y = target(x)
plt.plot(x, y);

In [ ]:
optimizer = BayesianOptimization(target, {'x': (-2, 6)}, random_state=1)

In [ ]:
optimizer.maximize(init_points=3, n_iter=0, acq='ei', kappa=0, xi=0.01)

In [ ]:
def posterior(optimizer, x_obs, y_obs, grid):
    optimizer._gp.fit(x_obs, y_obs)
    
    mu, sigma = optimizer._gp.predict(grid, return_std=True)
    return mu, sigma

In [ ]:
def plot_target(x, y):
    fig = plt.figure(figsize=(16, 10))
    steps = len(optimizer.space)
    
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
    axis = plt.subplot(gs[0])
    acq = plt.subplot(gs[1])
    
    axis.plot(x, y, linewidth=3, label='Objective Function')
    
    axis.set_xlim((-2.1, 6.1))
    axis.set_ylim((-0.6, 2.2))
    axis.set_ylabel('f(x)', fontdict={'size': 20})
    axis.set_yticks(np.arange(5)*0.5)
    axis.set_yticklabels(np.arange(5)*0.25, fontdict={'size': 15})
    axis.set_xlabel('x (learning rate)', fontdict={'size': 20})
    axis.set_xticks(np.arange(-2, 7))
    axis.set_xticklabels(np.arange(1, 10)*0.01, fontdict={'size': 15})
    
    acq.set_ylim((None, None))
    acq.set_yticks([])
    acq.set_yticklabels(np.arange(4)*2, fontdict={'size': 15})
    acq.set_ylabel('EI(x)', fontdict={'size': 20})
    acq.set_xlim((-2.1, 6.1))
    acq.set_xlabel('x (learning rate)', fontdict={'size': 20})
    acq.set_xticks(np.arange(-2, 7))
    acq.set_xticklabels(np.arange(1, 10)*0.01, fontdict={'size': 15})
    
    axis.legend(loc=1, borderaxespad=0., fontsize='large')

In [ ]:
def plot_gp(optimizer, x, y):
    fig = plt.figure(figsize=(16, 10))
    steps = len(optimizer.space)
    
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
    axis = plt.subplot(gs[0])
    acq = plt.subplot(gs[1])
    
    x_obs = np.array([[res["params"]["x"]] for res in optimizer.res])
    y_obs = np.array([res["target"] for res in optimizer.res])
    
    mu, sigma = posterior(optimizer, x_obs, y_obs, x)
    axis.plot(x, y, linewidth=3, label='Objective Function')
    axis.plot(x_obs.flatten(), y_obs, 'D', markersize=8, label=u'Observations', color='r')
    axis.plot(x, mu, '--', linewidth=2, color='k', label='Prediction')

    axis.fill(np.concatenate([x, x[::-1]]), 
              np.concatenate([mu - sigma, (mu + sigma)[::-1]]),
        alpha=.6, fc='c', ec='None', label='Uncertain area')
    
    axis.set_xlim((-2.1, 6.1))
    axis.set_ylim((-0.6, 2.2))
    axis.set_ylabel('f(x)', fontdict={'size':20})
    axis.set_yticks(np.arange(5)*0.5)
    axis.set_yticklabels(np.arange(5)*0.25, fontdict={'size': 15})
    axis.set_xlabel('x (learning rate)', fontdict={'size':20})
    axis.set_xticks(np.arange(-2, 7))
    axis.set_xticklabels(np.arange(1, 10)*0.01, fontdict={'size': 15})
    
    utility_function = UtilityFunction(kind="ei", kappa=0, xi=0.01)
    utility = utility_function.utility(x, optimizer._gp, optimizer._space.target.max())
    acq.plot(x, utility, label='Acquisition Function', color='purple')
    acq.plot(x[np.argmax(utility)], np.max(utility), '*', markersize=15, 
             label=u'Next Candidate', markerfacecolor='gold', markeredgecolor='k', markeredgewidth=1)
    acq.set_ylim((0, np.max(utility)*1.1))
    acq.set_yticks([])
    acq.set_ylabel('EI(x)', fontdict={'size': 20})
    acq.set_xlim((-2.1, 6.1))
    acq.set_xlabel('x (learning rate)', fontdict={'size': 20})
    acq.set_xticks(np.arange(-2, 7))
    acq.set_xticklabels(np.arange(1, 10)*0.01, fontdict={'size': 15})
    
    axis.legend(loc=1, borderaxespad=0., fontsize='large')
    acq.legend(loc=4, borderaxespad=0., fontsize='large')

In [ ]:
dst_dir = '../images/bayesian-optimization-overview-1/bayesian-optimization-process'

In [ ]:
plot_target(x, y)
plt.savefig(osp.join(dst_dir, 'bayesian-optimization-process-0.png'))

In [ ]:
plot_gp(optimizer, x, y)
plt.savefig(osp.join(dst_dir, 'bayesian-optimization-process-1-3.png'))

In [ ]:
for rep in range(3, 11):
    optimizer.maximize(init_points=0, n_iter=1, acq='ei', kappa=0, xi=0.01)
    plot_gp(optimizer, x, y)
    plt.savefig(osp.join(dst_dir, 'bayesian-optimization-process-{}.png'.format(rep+1)))
    plt.close()